In [89]:
import pandas as pd

# Data Loading

In [90]:
df = pd.read_csv('../data/GGWP-Toxic-Behavior/data/labeled/combined.csv')

In [91]:
df.head()

,index,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,real faker tweet,0,0,0,0,0,0
1,1,Lmfao,0,0,0,0,0,0
2,2,"bruh, the best teemo player from my country is...",0,0,0,0,0,0
3,3,me and nunu tiered 3 simped for adc,0,0,0,0,0,0
4,4,yeah but i think since seraphine was released ...,0,0,0,0,0,0


In [92]:
for col in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    print(f"{col}: {len(df[df[col] == 1])}")

print(f"clean: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)])}")

toxic: 562
severe_toxic: 4
obscene: 262
threat: 7
insult: 137
identity_hate: 11
clean: 2383


In [93]:
print(f"toxic: {len(df[(df['toxic'] == 1) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)])}")
print(f"severe_toxic: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 1) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)])}")
print(f"obscene: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 1) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)])}")
print(f"threat: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 1) & (df['insult'] == 0) & (df['identity_hate'] == 0)])}")
print(f"insult: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 1) & (df['identity_hate'] == 0)])}")
print(f"identity_hate: {len(df[(df['toxic'] == 0) & (df['severe_toxic'] == 0) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 1)])}")

toxic: 230
severe_toxic: 0
obscene: 0
threat: 0
insult: 0
identity_hate: 0


# Data Labeling with One-hot Encoding

In [94]:
# Convert separate columns to one-hot encoding
df['label'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
# Remove unused columns
df = df.drop(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [95]:
print(f"Before: {len(df)}")
# Remove mixed labels
df = df[df['label'].apply(lambda x: sum(x) == 0 or sum(x) == 1)]
print(f"After: {len(df)}")

Before: 2945
After: 2613


In [96]:
# Convert one-hot encoding to string labels
df['label'] = df['label'].apply(
    lambda x: 'clean' if x == [0, 0, 0, 0, 0, 0] else
    'toxic' if x == [1, 0, 0, 0, 0, 0] else
    'severe_toxic' if x == [0, 1, 0, 0, 0, 0] else
    'obscene' if x == [0, 0, 1, 0, 0, 0] else
    'threat' if x == [0, 0, 0, 1, 0, 0] else
    'insult' if x == [0, 0, 0, 0, 1, 0] else
    'identity_hate' if x == [0, 0, 0, 0, 0, 1] else
    "mixed")

In [97]:
df['label'].value_counts()

label
clean    2383
toxic     230
Name: count, dtype: int64

# Sampling
Sample 200 from each class, since the dataset is imbalanced.

In [98]:
df = df.groupby('label').apply(lambda x: x.sample(200, random_state=42))
df = df.reset_index(drop=True)

In [99]:
df.head()

,index,text,label
0,646,are you twitch single?,clean
1,2769,"@drsoils no need to be sorry, I liked it. LOL",clean
2,2097,"My sens is irrelevant to you, do what's comfor...",clean
3,2389,kekwwwwwwww,clean
4,2045,noice team work forsenCD,clean


In [100]:
df.to_csv('../data/sampled.csv', index=False)